# Deep Dive into LangChain

In [2]:
pip install -r ./requirements.txt -q

Note: you may need to restart the kernel to use updated packages.


In [3]:
# pip show openai

Name: openai
Version: 1.27.0
Summary: The official Python library for the openai API
Home-page: 
Author: 
Author-email: OpenAI <support@openai.com>
License: 
Location: /home/rasyidannas/anaconda3/lib/python3.11/site-packages
Requires: anyio, distro, httpx, pydantic, sniffio, tqdm, typing-extensions
Required-by: langchain-openai
Note: you may need to restart the kernel to use updated packages.


## Python-dotenv

In [8]:
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

# os.environ.get('OPENAI_API_KEY')

True

In [20]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()  

# invoking the llm (running the prompt)
output = llm.invoke('Explain quantum mechanics in one sentence.', model='gpt-3.5-turbo', temperature=0.1)
print(output.content)

Quantum mechanics is the branch of physics that describes the behavior of particles at the smallest scales, where traditional laws of physics no longer apply and instead, phenomena such as superposition and entanglement occur.


In [11]:
# this is for show detail our configuration CHatOpenAI
# help(ChatOpenAI)

In [9]:
from langchain.schema import (
    SystemMessage,
    AIMessage,
    HumanMessage
)

messages = [
    SystemMessage(content='You are a physicist and respond only in Indonesian'),
    HumanMessage(content='Explain quantum mechanics in one sentence.')
]

output = llm.invoke(messages)
print(output.content)

Mekanika kuantum adalah cabang fisika yang mempelajari perilaku partikel subatomik dan sistem mikroskopis melalui prinsip superposisi, keterkaitan, dan ketidakpastian.


## Caching LLM Responses

### In-Memory Cache

In [10]:
from langchain.globals import set_llm_cache
from langchain_openai import OpenAI
llm = OpenAI(model_name='gpt-3.5-turbo-instruct')

In [11]:
%%time
from langchain.cache import  InMemoryCache
set_llm_cache(InMemoryCache())
prompt = 'Tell me a joke that a toddler can understand'
llm.invoke(prompt)

CPU times: user 645 ms, sys: 84.1 ms, total: 729 ms
Wall time: 3.03 s


"\n\nWhy did the banana go to the doctor?\n\nBecause it wasn't peeling well!"

In [12]:
%%time
llm.invoke(prompt)

CPU times: user 1.02 ms, sys: 0 ns, total: 1.02 ms
Wall time: 1.04 ms


"\n\nWhy did the banana go to the doctor?\n\nBecause it wasn't peeling well!"

### SQLite Caching

In [13]:
from langchain.cache import SQLiteCache
set_llm_cache(SQLiteCache(database_path='.langchain.db'))

# First request (not in cahce, takes Longer)
llm.invoke('Tell me a joke')

# Second request (cached, faster)
llm.invoke('Tell me a joke')

"\n\nWhy don't scientists trust atoms?\n\nBecause they make up everything."

## LLM Streaming

In [14]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()
prompt = 'Write a rock song about the Moon and a Raven.'
print(llm.invoke(prompt).content)

Verse 1:
Beneath the moonlit sky
A raven takes flight
Its wings black as night
Guided by the pale moonlight

Chorus:
Moon and raven, dance in the night
Their spirits intertwined, a mystical sight
They soar through the darkness, side by side
In the shadow of the moon, they will abide

Verse 2:
The raven calls out
In a haunting, eerie cry
Its message carried on the wind
As the moon watches from on high

Chorus:
Moon and raven, dance in the night
Their spirits intertwined, a mystical sight
They soar through the darkness, side by side
In the shadow of the moon, they will abide

Bridge:
The moon whispers secrets
To the raven in flight
Together they roam
Through the endless night

Chorus:
Moon and raven, dance in the night
Their spirits intertwined, a mystical sight
They soar through the darkness, side by side
In the shadow of the moon, they will abide

Outro:
Moon and raven, forever bound
In the darkness, their presence is found
A mysterious pair, in the velvet sky
Moon and raven, flying hi

In [15]:
### Enable streaming
for chunk in llm.stream(prompt):
    print(chunk.content, end='', flush=True)

Verse 1:
Underneath the midnight sky
The moon shines bright up high
A raven calls out in the night
Its haunting cry fills me with fright

Chorus:
Moon and raven, guardians of the night
Their spirits intertwined, a mystical sight
They watch over us, with wisdom and might
Moon and raven, shining in the moonlight

Verse 2:
The raven flies with grace and poise
Its black feathers shimmer in the noise
The moon's glow guides its path
As it soars through the aftermath

Chorus:
Moon and raven, guardians of the night
Their spirits intertwined, a mystical sight
They watch over us, with wisdom and might
Moon and raven, shining in the moonlight

Bridge:
In the darkness, they reign supreme
In their realm, they create a dream
A world of mystery and wonder
Where the moon and raven never sunder

Chorus:
Moon and raven, guardians of the night
Their spirits intertwined, a mystical sight
They watch over us, with wisdom and might
Moon and raven, shining in the moonlight

Outro:
As the night fades into day


## PromptTemplates

In [16]:
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

template = '''You are an experience virologist. Write a few sentences about the following virus "{virus}" in {language}.'''
prompt_template = PromptTemplate.from_template(template=template)

prompt = prompt_template.format(virus='hiv', language='german')
prompt


'You are an experience virologist. Write a few sentences about the following virus "hiv" in german.'

In [17]:
llm = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0)
output = llm.invoke(prompt)
print(output.content)

HIV, das humane Immundefizienzvirus, ist ein Virus, das das Immunsystem des Menschen schwächt und zu AIDS führen kann. Es wird hauptsächlich durch ungeschützten Geschlechtsverkehr, den Austausch von infizierten Nadeln oder von der Mutter auf das Kind während der Schwangerschaft übertragen. Es ist wichtig, sich über die Übertragungswege und Präventionsmaßnahmen von HIV zu informieren, um die Verbreitung dieser lebensbedrohlichen Krankheit einzudämmen.


### ChatPromptTemplates

In [18]:
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain_core.messages import SystemMessage

chat_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(content='You respond only in the JSON format.'),
        HumanMessagePromptTemplate.from_template('Top {n} countries in {area} by population.')
    ]
)

messages = chat_template.format_messages(n='10', area='World')
print(messages)

[SystemMessage(content='You respond only in the JSON format.'), HumanMessage(content='Top 10 countries in World by population.')]


In [19]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI()
output = llm.invoke(messages)
print(output.content)

{
    "1": {
        "country": "China",
        "population": "1,439,323,776"
    },
    "2": {
        "country": "India",
        "population": "1,380,004,385"
    },
    "3": {
        "country": "United States",
        "population": "331,002,651"
    },
    "4": {
        "country": "Indonesia",
        "population": "273,523,615"
    },
    "5": {
        "country": "Pakistan",
        "population": "220,892,331"
    },
    "6": {
        "country": "Brazil",
        "population": "212,559,417"
    },
    "7": {
        "country": "Nigeria",
        "population": "206,139,589"
    },
    "8": {
        "country": "Bangladesh",
        "population": "164,689,383"
    },
    "9": {
        "country": "Russia",
        "population": "145,934,462"
    },
    "10": {
        "country": "Mexico",
        "population": "128,932,753"
    }
}


## Simple Chains

In [20]:
from langchain_openai import ChatOpenAI
from langchain import PromptTemplate
from langchain.chains import LLMChain

llm = ChatOpenAI()
template = '''You are an experience virologist. Write a few sentences about the following virus "{virus}" in {language}.'''
prompt_template = PromptTemplate.from_template(template=template)

chain = LLMChain(
    llm=llm,
    prompt=prompt_template,
    verbose=True
)

output = chain.invoke({'virus': 'HSV', 'language': 'Indonesian'})



> Entering new LLMChain chain...
Prompt after formatting:
You are an experience virologist. Write a few sentences about the following virus "HSV" in Indonesian.

> Finished chain.


/home/rasyidannas/anaconda3/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [22]:
print(output)

{'virus': 'HSV', 'language': 'Indonesian', 'text': 'HSV atau herpes simpleks virus adalah virus yang menyebabkan infeksi pada manusia. Virus ini dapat menyebabkan luka di mulut (herpes labialis) atau di area genital (herpes genitalis). HSV sangat menular dan dapat menyebar melalui kontak langsung dengan luka yang terinfeksi. Penyakit ini dapat kambuh secara periodik dan tidak memiliki obat yang dapat menyembuhkan secara permanen.'}


In [23]:
template = 'What is the capital of {country}?. List the top 3 places to visit in that city. Use bullet points'
prompt_template = PromptTemplate.from_template(template=template)

chain = LLMChain(
    llm=llm,
    prompt=prompt_template,
    verbose=True
)

country = input('Enter Country: ')
output = chain.invoke(country)
print(output['text'])

Enter Country:  Indonesia




> Entering new LLMChain chain...
Prompt after formatting:
What is the capital of Indonesia?. List the top 3 places to visit in that city. Use bullet points

> Finished chain.
The capital of Indonesia is Jakarta.

Top 3 places to visit in Jakarta:
- National Monument (Monas)
- Taman Mini Indonesia Indah
- Kota Tua (Old Town)


## Sequential Chains

In [10]:
from langchain_openai import ChatOpenAI
from langchain import PromptTemplate
from langchain.chains import LLMChain, SimpleSequentialChain

llm1 = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0.5)
prompt_template1 = PromptTemplate.from_template(
    template='You are an experiment scientist and Python programmer. Write a function that implements the concept of {concept}.'
)

chain1 = LLMChain(llm=llm1, prompt=prompt_template1)

llm2 = ChatOpenAI(model_name='gpt-4-turbo-preview', temperature=1.2)
prompt_template2 = PromptTemplate.from_template(
    template='Given the python function {function}, describe it as detailed as possible.'
)
chain2 = LLMChain(llm=llm2, prompt=prompt_template2)

overall_chain = SimpleSequentialChain(chains=[chain1, chain2], verbose=True)
output = overall_chain.invoke('linear regression')

/home/rasyidannas/anaconda3/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(




> Entering new SimpleSequentialChain chain...
Sure! Here is an example of a Python function that implements linear regression:

```python
import numpy as np

def linear_regression(X, y):
    # Add a column of ones to X for the intercept term
    X = np.column_stack((np.ones(len(X)), X))
    
    # Calculate the coefficients using the normal equation
    theta = np.linalg.inv(X.T.dot(X)).dot(X.T).dot(y)
    
    return theta

# Example usage
X = np.array([1, 2, 3, 4, 5])
y = np.array([2, 4, 6, 8, 10])

theta = linear_regression(X, y)
print("Coefficients:", theta)
```

This function takes in an array `X` of input values and an array `y` of output values, and calculates the coefficients for a linear regression model using the normal equation. The function returns the coefficients as a numpy array.

You can use this function to perform linear regression on your data and obtain the coefficients for the best-fit line.
The provided Python function, named `linear_regression`, is designed to 

In [27]:
print(output['output'])

The Python function you've shared, named `linear_regression`, computes the slope and intercept for a best-fit straight line through a set of data points, based on the method of least squares. This process is fundamental for linear regression analysis, which is widely used in statistics and data science to model linear relationships between variables. Here's a detailed explanation of how it works:

### Import Statement
```python
import numpy as np
```
- The function begins by importing the NumPy library, which provides support for arrays (like sequences in Python, but more generally a multi-dimensional matrix), along with a collection of mathematical functions to operate on these arrays. NumPy is highly efficient for numerical computations.

### Function Definition
```python
def linear_regression(x, y):
```
- The function `linear_regression` is defined with two parameters: `x` and `y`. These are arrays representing the independent (predictor) and dependent (outcome) variables, respectiv

## LangChain Agents in Action: Python REPL

In [28]:
pip install -q langchain_experimental

Note: you may need to restart the kernel to use updated packages.


In [29]:
from langchain_experimental.utilities import PythonREPL
python_repl = PythonREPL()
python_repl.run('print([n for n in range (1, 100) if n % 13 == 0])')

Python REPL can execute arbitrary code. Use with caution.


'[13, 26, 39, 52, 65, 78, 91]\n'

In [10]:
from langchain_experimental.agents.agent_toolkits import create_python_agent
from langchain_experimental.tools.python.tool import PythonREPLTool
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-4-turbo-preview', temperature=0)
agent_executor = create_python_agent(
    llm=llm,
    tool=PythonREPLTool(), #this is tool in necessary for agent
    verbose=True
)

agent_executor.invoke('Calculate the square root of the factorial of 12 and display it with 4 decimal points')




> Entering new AgentExecutor chain...


Python REPL can execute arbitrary code. Use with caution.


To solve this, I will first calculate the factorial of 12 using the math module in Python. Then, I will calculate the square root of that result. Finally, I will format the output to display it with 4 decimal points. Let's start by importing the math module and calculating the factorial of 12.
Action: Python_REPL
Action Input: import math
Observation: 
Thought:Now that the math module is imported, I can calculate the factorial of 12.
Action: Python_REPL
Action Input: factorial_12 = math.factorial(12)
Observation: 
Thought:Now that I have the factorial of 12, I will calculate its square root.
Action: Python_REPL
Action Input: sqrt_factorial_12 = math.sqrt(factorial_12)
Observation: 
Thought:Now that I have the square root of the factorial of 12, I will format it to display with 4 decimal points.
Action: Python_REPL
Action Input: print(f"{sqrt_factorial_12:.4f}")
Observation: 21886.1052

Thought:I now know the final answer
Final Answer: 21886.1052

> Finished chain.


{'input': 'Calculate the square root of the factorial of 12 and display it with 4 decimal points',
 'output': '21886.1052'}

In [11]:
response = agent_executor.invoke('What is the answer to 5.1 ** 7.3?')



> Entering new AgentExecutor chain...
I need to calculate 5.1 raised to the power of 7.3 to get the answer.
Action: Python_REPL
Action Input: print(5.1 ** 7.3)
Observation: 146306.05007233328

Thought:I now know the final answer
Final Answer: 146306.05007233328

> Finished chain.


In [33]:
response

{'input': 'What is the answer to 5.1 ** 7.3?', 'output': '146306.05007233328'}

In [35]:
print(response['input'])

What is the answer to 5.1 ** 7.3?


In [37]:
print(response['output'])

146306.05007233328


## LangChain Tools: DuckDuckGo and Wikipedia

In [16]:
pip install -q duckduckgo-search

Note: you may need to restart the kernel to use updated packages.


In [28]:
# optional, filter workings (recommended in production)
import warnings
warnings.filterwarnings('ignore', module='langchain')

In [29]:
from langchain.tools import DuckDuckGoSearchRun

search = DuckDuckGoSearchRun()
output = search.invoke('Where was Freddie Mercury born?')
print(output)

RatelimitException: https://links.duckduckgo.com/d.js?q=Where+was+Freddie+Mercury+born%3F&kl=wt-wt&l=wt-wt&p=&s=0&df=y&vqd=4-163719682109771439209838459638209803915&ex=-1 202 Ratelimit

In [13]:
search.name

'duckduckgo_search'

In [14]:
search.description

'A wrapper around DuckDuckGo Search. Useful for when you need to answer questions about current events. Input should be a search query.'

In [21]:
from langchain.tools import DuckDuckGoSearchResults
search = DuckDuckGoSearchResults()
output = search.run('Freddie Mercury and Queen.')
print(output)

RatelimitException: https://links.duckduckgo.com/d.js?q=Freddie+Mercury+and+Queen.&kl=wt-wt&l=wt-wt&p=&s=0&df=y&vqd=4-61464889412872897942089859488741580185&ex=-1 202 Ratelimit

In [18]:
from langchain_community.utilities import DuckDuckGoSearchAPIWrapper

wrapper = DuckDuckGoSearchAPIWrapper(region='de-de', max_results=3, safesearch='moderate')
search = DuckDuckGoSearchResults(api_wrapper=wrapper, source='news')
output = search.run('Berlin')

RatelimitException: https://links.duckduckgo.com/d.js?q=Berlin&kl=de-de&l=de-de&p=&s=0&df=y&vqd=4-58725434474055168076950146975429701280&ex=-1 202 Ratelimit

In [19]:
print(output)

content='Quantum mechanics is the branch of physics that describes the behavior of matter and energy on the smallest scales, where particles can exist in multiple states simultaneously and are governed by probabilistic laws.' response_metadata={'token_usage': {'completion_tokens': 37, 'prompt_tokens': 15, 'total_tokens': 52}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-ebb05cf2-da54-47a1-8b43-ee3d75c909a0-0'


In [22]:
import re
pattern = r'snippet: (.*?), title: (.*?), link: (.*?)\],'
matches = re.findall(pattern, output, re.DOTALL)

for snippet, title, link in matches:
    print(f'Snippet: {snippet}\nTitle: {title}\nLink: {link}\n')
    print('-' * 50)

TypeError: expected string or bytes-like object, got 'AIMessage'

### Wikipedia

In [23]:
pip install -q wikipedia

Note: you may need to restart the kernel to use updated packages.


In [24]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [25]:
api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=10000)
wiki = WikipediaQueryRun(api_wrapper=api_wrapper)
wiki.invoke({'query': 'llamaindex'})

ConnectionError: ('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer'))

In [26]:
output = wiki.invoke('Google Gemini')

ConnectionError: ('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer'))

In [27]:
print(output)

content='Quantum mechanics is the branch of physics that describes the behavior of particles at the smallest scales, where traditional laws of physics no longer apply and instead, phenomena such as superposition and entanglement occur.' response_metadata={'token_usage': {'completion_tokens': 41, 'prompt_tokens': 15, 'total_tokens': 56}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-5425a5d8-b4f2-4d02-b7af-73a43624c06c-0'


## Creating a ReAct Agent

In [13]:
pip install -q langchainhub 

Note: you may need to restart the kernel to use updated packages.


In [3]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

True

In [14]:
from langchain.prompts import PromptTemplate
from langchain import hub
from langchain.agents import Tool, AgentExecutor, initialize_agent, create_react_agent
from langchain.tools import DuckDuckGoSearchRun, WikipediaQueryRun
from langchain.utilities import WikipediaAPIWrapper
from langchain_experimental.tools.python.tool import PythonREPLTool
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model_name='gpt-4-turbo-preview', temperature=0)

template = '''
Answer the following questions in GERMAN as best you can.
Question: {q}
'''

prompt_template = PromptTemplate.from_template(template)
prompt = hub.pull('hwchase17/react')
# print(type(prompt))
# print(prompt.input_variables)
# print(prompt.template)

# 1. Python REPL Tool (for executing Python code)
python_repl = PythonREPLTool()
python_repl_tool = Tool(
    name='Python REPL',
    func=python_repl.run,
    description='Useful when you need to use Python to answer a question. You should input Python code.'
)

# 2. Wikipedia Tool (for searching Wikipedia)
api_wrapper = WikipediaAPIWrapper()
wikipedia = WikipediaQueryRun(api_wrapper=api_wrapper)
wikipedia_tool = Tool(
    name='Wikipedia',
    func=wikipedia.run,
    description='Useful for when you need to look up a topic, country, or person on Wikipedia.'
)

# 3. DuckDuckGo Search Tool (for general web searches)
search = DuckDuckGoSearchRun()
duckduckgo_tool = Tool(
    name='DuckDuckGo Search',
    func=search.run,
    description='Useful for when you need to perform an internet search to find information that another tool can\'t provide.'
)

# Combine the tools into a list
tools = [python_repl_tool, wikipedia_tool, duckduckgo_tool]

# Create a react agent with the ChatOpenAI model, tools, and prompt
agent = create_react_agent(llm, tools, prompt)

# Initialize the AgentExecutor
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True,
    max_iterations=10
)

In [15]:
question = 'Generate the first 20 numbers in the Fibonanci series.'
output = agent_executer.invoke({
    'input': prompt_template.format(q=question)
})

NameError: name 'agent_executer' is not defined

In [16]:
print(output['input'])

linear regression


In [18]:
output['output']

"The provided Python function, named `linear_regression`, is designed to calculate the coefficients of a simple linear regression model given a set of input (independent) variables, `X`, and output (dependent) variables, `y`. Linear regression is a fundamental statistical method used to model the relationship between a dependent variable and one or more independent variables by fitting a linear equation to observed data. The specific technique implemented in this function leverages the normal equation method for linear regression.\n\nHere's a step-by-step explanation of how the function works:\n\n1. **Importing Required Module:**\n   The function begins by importing `numpy`, a foundational package for scientific computing in Python. It offers powerful data structures, implementing multi-dimensional arrays and matrices, along with a collection of mathematical functions to operate on these arrays.\n\n2. **Function Definition:**\n   The `linear_regression` function is defined to take two 

In [19]:
question = 'Who is the current prime minister of the UK?'
output = agent_executor.invoke({
    'input': prompt_template.format(q=question)
})



> Entering new AgentExecutor chain...
I need to find out who the current Prime Minister of the United Kingdom is as of my last update in 2023.

Action: DuckDuckGo Search
Action Input: current prime minister of the UK 2023

RatelimitException: https://links.duckduckgo.com/d.js?q=current+prime+minister+of+the+UK+2023&kl=wt-wt&l=wt-wt&p=&s=0&df=y&vqd=4-216470128803045813035160287156500387244&ex=-1 202 Ratelimit